# 第 4 章：梯度消失与梯度爆炸

> 理解 RNN 训练的核心难题，为 LSTM/GRU 做铺垫

---

## 本章目标

完成本章后，你将能够：

- [ ] 理解为什么 RNN 会出现梯度消失/爆炸
- [ ] 用数学分析梯度的指数衰减/增长
- [ ] 可视化长序列中的梯度行为
- [ ] 掌握缓解梯度问题的基本技术
- [ ] 理解 LSTM/GRU 设计的必要性

---

## 问题引入：RNN 的"健忘症"

让我们先看一个现象：RNN 似乎无法记住长期信息

```
句子 1（短期依赖）：
"The cat sat on the mat." → 预测 "mat" 只需要看 "sat on the"

句子 2（长期依赖）：
"I grew up in France... [100 words later]... I speak fluent ___"
→ 预测 "French" 需要记住 100 词之前的 "France"
```

为什么 RNN 难以处理长期依赖？答案就在梯度中！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

print("环境准备完成！")

---

## 第一部分：梯度消失/爆炸的数学分析

### 1.1 回顾 BPTT 中的梯度链

RNN 的隐状态更新：
$$h_t = \tanh(W_{hh} \cdot h_{t-1} + W_{xh} \cdot x_t + b_h)$$

从时间 $t$ 回传到时间 $k$ 的梯度（$t > k$）：

$$\frac{\partial h_t}{\partial h_k} = \prod_{i=k+1}^{t} \frac{\partial h_i}{\partial h_{i-1}}$$

其中每一项：
$$\frac{\partial h_i}{\partial h_{i-1}} = \text{diag}(1 - h_i^2) \cdot W_{hh}$$

所以：
$$\frac{\partial h_t}{\partial h_k} = \prod_{i=k+1}^{t} \text{diag}(1 - h_i^2) \cdot W_{hh}$$

### 1.2 关键洞察：矩阵乘积的范数

考虑连续乘以同一个矩阵 $W$ 共 $n$ 次：

$$W^n$$

这个乘积的行为取决于 $W$ 的**特征值**：

设 $W$ 的最大特征值为 $\lambda_{max}$：

| $|\lambda_{max}|$ | 效果 | 结果 |
|-------------------|------|------|
| $< 1$ | 指数衰减 | 梯度消失 |
| $= 1$ | 稳定 | 理想 |
| $> 1$ | 指数增长 | 梯度爆炸 |

**更糟糕的是**：RNN 中还有 tanh 的导数 $(1 - h^2)$，它的值在 $[0, 1]$ 之间，进一步加剧了梯度消失！

In [ ]:
def visualize_matrix_power():
    """
    可视化矩阵连乘的效果：
    展示不同特征值对梯度的影响
    """
    print("=" * 60)
    print("矩阵连乘效果演示")
    print("=" * 60)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    n_steps = 50
    
    # ========================================
    # 情况 1: |λ| < 1 → 梯度消失
    # ========================================
    ax = axes[0]
    
    # 创建特征值 < 1 的矩阵
    # 使用对角矩阵便于分析
    eigenvalues_small = [0.9, 0.8, 0.7]
    
    for ev in eigenvalues_small:
        W = ev * np.eye(2)  # 简单的对角矩阵
        norms = []
        W_power = np.eye(2)
        
        for t in range(n_steps):
            W_power = W_power @ W
            norms.append(np.linalg.norm(W_power))
        
        ax.plot(range(n_steps), norms, label=f'λ = {ev}')
    
    ax.set_xlabel('时间步', fontsize=12)
    ax.set_ylabel('||W^t||', fontsize=12)
    ax.set_title('梯度消失: |λ| < 1', fontsize=14, color='blue')
    ax.legend()
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=1e-10, color='red', linestyle='--', label='消失阈值')
    
    # ========================================
    # 情况 2: |λ| = 1 → 稳定
    # ========================================
    ax = axes[1]
    
    eigenvalues_one = [1.0, 0.99, 1.01]
    
    for ev in eigenvalues_one:
        W = ev * np.eye(2)
        norms = []
        W_power = np.eye(2)
        
        for t in range(n_steps):
            W_power = W_power @ W
            norms.append(np.linalg.norm(W_power))
        
        ax.plot(range(n_steps), norms, label=f'λ = {ev}')
    
    ax.set_xlabel('时间步', fontsize=12)
    ax.set_ylabel('||W^t||', fontsize=12)
    ax.set_title('边界情况: |λ| ≈ 1', fontsize=14, color='green')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # ========================================
    # 情况 3: |λ| > 1 → 梯度爆炸
    # ========================================
    ax = axes[2]
    
    eigenvalues_large = [1.1, 1.2, 1.3]
    
    for ev in eigenvalues_large:
        W = ev * np.eye(2)
        norms = []
        W_power = np.eye(2)
        
        for t in range(n_steps):
            W_power = W_power @ W
            norms.append(np.linalg.norm(W_power))
        
        ax.plot(range(n_steps), norms, label=f'λ = {ev}')
    
    ax.set_xlabel('时间步', fontsize=12)
    ax.set_ylabel('||W^t||', fontsize=12)
    ax.set_title('梯度爆炸: |λ| > 1', fontsize=14, color='red')
    ax.legend()
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 打印具体数值
    print("\n具体数值示例:")
    print("-" * 40)
    for ev, name in [(0.9, "消失"), (1.0, "稳定"), (1.1, "爆炸")]:
        result = ev ** 50
        print(f"λ={ev:.1f}, t=50: λ^t = {result:.2e} ({name})")


visualize_matrix_power()

### 1.3 tanh 导数的影响

tanh 的导数：
$$\frac{d\tanh(x)}{dx} = 1 - \tanh^2(x)$$

这个导数有什么特点？

In [ ]:
def visualize_tanh_derivative():
    """
    可视化 tanh 及其导数
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    x = np.linspace(-4, 4, 200)
    
    # 左图：tanh 函数
    ax = axes[0]
    y = np.tanh(x)
    ax.plot(x, y, 'b-', linewidth=2, label='tanh(x)')
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
    ax.axhline(y=1, color='r', linestyle='--', alpha=0.5)
    ax.axhline(y=-1, color='r', linestyle='--', alpha=0.5)
    ax.fill_between(x, -1, 1, alpha=0.1, color='blue')
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('tanh(x)', fontsize=12)
    ax.set_title('tanh 函数: 输出被压缩到 [-1, 1]', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-1.5, 1.5)
    
    # 右图：tanh 导数
    ax = axes[1]
    dy = 1 - np.tanh(x) ** 2  # tanh 的导数
    ax.plot(x, dy, 'r-', linewidth=2, label="tanh'(x) = 1 - tanh²(x)")
    ax.axhline(y=1, color='g', linestyle='--', alpha=0.5, label='最大值 = 1')
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax.axhline(y=0.25, color='orange', linestyle='--', alpha=0.5, label='当 |x|=1 时约 0.42')
    ax.fill_between(x, 0, dy, alpha=0.2, color='red')
    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel("tanh'(x)", fontsize=12)
    ax.set_title('tanh 导数: 总是 ≤ 1, 远离原点时趋近 0', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.1, 1.2)
    
    plt.tight_layout()
    plt.show()
    
    # 关键点分析
    print("\n关键观察:")
    print("-" * 50)
    print("1. tanh'(x) 的最大值是 1（在 x=0 处）")
    print("2. 当 |x| > 1 时，tanh'(x) < 0.42")
    print("3. 当 |x| > 2 时，tanh'(x) < 0.07")
    print("4. 当 |x| > 3 时，tanh'(x) < 0.01")
    print("\n结论: tanh 的导数大部分时候远小于 1，这会加剧梯度消失！")


visualize_tanh_derivative()

### 1.4 组合效应：完整的梯度分析

BPTT 中从时间 $t$ 到时间 $k$ 的梯度：

$$\frac{\partial h_t}{\partial h_k} = \prod_{i=k+1}^{t} \underbrace{\text{diag}(1 - h_i^2)}_{\leq 1} \cdot W_{hh}$$

每一项的范数大致为：
$$\|\text{diag}(1 - h_i^2) \cdot W_{hh}\| \approx \|W_{hh}\| \cdot \bar{\sigma}'$$

其中 $\bar{\sigma}'$ 是平均的 tanh 导数（通常 < 1）。

经过 $(t-k)$ 步后：
$$\left\|\frac{\partial h_t}{\partial h_k}\right\| \approx \left(\|W_{hh}\| \cdot \bar{\sigma}'\right)^{t-k}$$

**这是一个指数函数！**

In [ ]:
def analyze_gradient_flow():
    """
    分析 RNN 中梯度流动的完整图景
    """
    print("=" * 60)
    print("RNN 梯度流动分析")
    print("=" * 60)
    
    # 不同 W_hh 范数下的梯度行为
    w_norms = [0.5, 0.8, 1.0, 1.2, 1.5]
    avg_tanh_deriv = 0.5  # 假设平均 tanh 导数
    
    seq_lens = np.arange(1, 101)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图：梯度范数随时间步的变化
    ax = axes[0]
    
    for w_norm in w_norms:
        effective_factor = w_norm * avg_tanh_deriv
        gradient_norms = effective_factor ** seq_lens
        
        label = f'||W||={w_norm}, 有效因子={effective_factor:.2f}'
        ax.semilogy(seq_lens, gradient_norms, label=label, linewidth=2)
    
    ax.axhline(y=1, color='k', linestyle='--', alpha=0.5)
    ax.axhline(y=1e-10, color='red', linestyle='--', alpha=0.5)
    ax.axhline(y=1e10, color='red', linestyle='--', alpha=0.5)
    
    ax.set_xlabel('时间步差 (t - k)', fontsize=12)
    ax.set_ylabel('梯度范数 (对数尺度)', fontsize=12)
    ax.set_title(f'梯度范数 vs 时间步 (平均 tanh\' = {avg_tanh_deriv})', fontsize=14)
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(1e-50, 1e50)
    
    # 添加区域标注
    ax.fill_between(seq_lens, 1e-50, 1e-10, alpha=0.1, color='blue', label='梯度消失区')
    ax.fill_between(seq_lens, 1e10, 1e50, alpha=0.1, color='red', label='梯度爆炸区')
    
    # 右图：有效梯度距离
    ax = axes[1]
    
    # 计算梯度衰减到特定阈值需要的时间步
    thresholds = [1e-3, 1e-5, 1e-10]
    effective_factors = np.linspace(0.3, 0.99, 50)
    
    for thresh in thresholds:
        # 求解: factor^n = thresh => n = log(thresh) / log(factor)
        steps_to_threshold = np.log(thresh) / np.log(effective_factors)
        ax.plot(effective_factors, steps_to_threshold, 
                label=f'梯度衰减到 {thresh}', linewidth=2)
    
    ax.set_xlabel('有效因子 (||W|| × avg tanh\')', fontsize=12)
    ax.set_ylabel('所需时间步数', fontsize=12)
    ax.set_title('梯度有效传播距离', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 200)
    
    plt.tight_layout()
    plt.show()
    
    # 具体数值示例
    print("\n具体示例:")
    print("-" * 50)
    print("假设 ||W_hh|| = 1.0, 平均 tanh' = 0.5")
    print("有效因子 = 0.5")
    print(f"")
    for steps in [10, 20, 50, 100]:
        grad = 0.5 ** steps
        print(f"  {steps} 步后梯度: 0.5^{steps} = {grad:.2e}")


analyze_gradient_flow()

---

## 第二部分：实验验证

### 2.1 实际 RNN 中的梯度观察

In [ ]:
def observe_gradient_in_rnn():
    """
    在实际 RNN 中观察梯度消失现象
    """
    print("=" * 60)
    print("实际 RNN 中的梯度观察")
    print("=" * 60)
    
    # 创建简单的 RNN
    class SimpleRNN(nn.Module):
        def __init__(self, input_size, hidden_size):
            super().__init__()
            self.hidden_size = hidden_size
            self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, 1)
        
        def forward(self, x):
            # 保存每个时间步的隐状态（用于分析梯度）
            out, _ = self.rnn(x)
            # 只用最后一个时间步的输出
            return self.fc(out[:, -1, :])
    
    # 测试不同序列长度
    seq_lengths = [10, 20, 50, 100, 200]
    hidden_size = 32
    input_size = 1
    
    gradient_norms = {}
    
    for seq_len in seq_lengths:
        torch.manual_seed(42)
        model = SimpleRNN(input_size, hidden_size)
        
        # 创建输入：在第一个时间步有特殊信号
        x = torch.zeros(1, seq_len, input_size)
        x[0, 0, 0] = 1.0  # 只在 t=0 有输入
        
        # 前向传播
        output = model(x)
        
        # 反向传播
        output.backward()
        
        # 获取 W_hh 的梯度
        # RNN 的权重包括 weight_hh_l0
        grad_norm = model.rnn.weight_hh_l0.grad.norm().item()
        gradient_norms[seq_len] = grad_norm
        
        print(f"序列长度 {seq_len:3d}: ||∂L/∂W_hh|| = {grad_norm:.6e}")
    
    # 可视化
    fig, ax = plt.subplots(figsize=(10, 5))
    
    ax.bar(range(len(seq_lengths)), list(gradient_norms.values()), 
           tick_label=[str(s) for s in seq_lengths], color='steelblue', edgecolor='navy')
    ax.set_xlabel('序列长度', fontsize=12)
    ax.set_ylabel('梯度范数', fontsize=12)
    ax.set_title('RNN 梯度范数 vs 序列长度', fontsize=14)
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3, axis='y')
    
    # 添加趋势说明
    ax.text(0.95, 0.95, '梯度随序列长度\n指数级衰减!', 
            transform=ax.transAxes, ha='right', va='top',
            fontsize=12, color='red',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    return gradient_norms


gradient_norms = observe_gradient_in_rnn()

### 2.2 逐时间步的梯度分析

In [ ]:
def analyze_gradient_per_timestep():
    """
    分析每个时间步对最终梯度的贡献
    """
    print("=" * 60)
    print("逐时间步梯度分析")
    print("=" * 60)
    
    # 创建手动实现的 RNN（便于提取每步梯度）
    seq_len = 50
    hidden_size = 16
    input_size = 1
    
    # 初始化参数
    torch.manual_seed(42)
    W_xh = torch.randn(hidden_size, input_size, requires_grad=True) * 0.1
    W_hh = torch.randn(hidden_size, hidden_size, requires_grad=True) * 0.1
    W_hy = torch.randn(1, hidden_size, requires_grad=True) * 0.1
    
    # 输入序列
    x = torch.zeros(seq_len, input_size)
    x[0, 0] = 1.0  # 只在 t=0 有输入
    
    # 前向传播，保存每个时间步的隐状态
    h_list = []
    h = torch.zeros(hidden_size)
    
    for t in range(seq_len):
        h = torch.tanh(W_xh @ x[t] + W_hh @ h)
        h_list.append(h)
    
    # 最终输出
    y = W_hy @ h_list[-1]
    
    # 计算每个隐状态对输出的梯度
    grad_norms = []
    
    for t in range(seq_len):
        # 清除之前的梯度
        if h_list[t].grad is not None:
            h_list[t].grad.zero_()
        
        # 计算 dy/dh_t
        grad = torch.autograd.grad(y, h_list[t], retain_graph=True)[0]
        grad_norms.append(grad.norm().item())
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图：线性尺度
    ax = axes[0]
    ax.plot(range(seq_len), grad_norms, 'b-', linewidth=2)
    ax.fill_between(range(seq_len), grad_norms, alpha=0.3)
    ax.set_xlabel('时间步 t', fontsize=12)
    ax.set_ylabel('||∂y/∂h_t||', fontsize=12)
    ax.set_title('各时间步隐状态对输出的梯度（线性尺度）', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    # 标注
    ax.annotate('越早的时间步\n梯度越小', xy=(10, grad_norms[10]), 
                xytext=(20, max(grad_norms)*0.5),
                arrowprops=dict(arrowstyle='->', color='red'),
                fontsize=11, color='red')
    
    # 右图：对数尺度
    ax = axes[1]
    ax.semilogy(range(seq_len), grad_norms, 'r-', linewidth=2)
    ax.fill_between(range(seq_len), grad_norms, alpha=0.3, color='red')
    ax.set_xlabel('时间步 t', fontsize=12)
    ax.set_ylabel('||∂y/∂h_t|| (对数尺度)', fontsize=12)
    ax.set_title('各时间步隐状态对输出的梯度（对数尺度）', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    # 拟合指数衰减
    from scipy.optimize import curve_fit
    def exp_decay(t, a, b):
        return a * np.exp(b * t)
    
    try:
        popt, _ = curve_fit(exp_decay, range(seq_len), grad_norms, p0=[1, -0.1])
        t_fit = np.linspace(0, seq_len-1, 100)
        ax.plot(t_fit, exp_decay(t_fit, *popt), 'g--', linewidth=2, 
                label=f'拟合: {popt[0]:.2f}×exp({popt[1]:.3f}t)')
        ax.legend()
        print(f"\n指数拟合: 衰减率 = {popt[1]:.4f}")
        print(f"每步衰减约 {np.exp(popt[1]):.4f} 倍")
    except:
        pass
    
    plt.tight_layout()
    plt.show()
    
    # 打印统计
    print("\n梯度统计:")
    print("-" * 40)
    print(f"t=0 (最远) 梯度: {grad_norms[0]:.6e}")
    print(f"t={seq_len//2} (中间) 梯度: {grad_norms[seq_len//2]:.6e}")
    print(f"t={seq_len-1} (最近) 梯度: {grad_norms[-1]:.6e}")
    print(f"\n衰减比例: {grad_norms[-1]/grad_norms[0]:.2e} 倍")
    
    return grad_norms


grad_per_step = analyze_gradient_per_timestep()

---

## 第三部分：长期依赖任务实验

### 3.1 设计一个需要长期记忆的任务

In [ ]:
def long_term_memory_task():
    """
    长期依赖任务：记住序列开头的信息
    
    任务设计:
    - 输入: [标记, 0, 0, 0, ..., 0, 0]
    - 输出: 预测开头的标记（+1 或 -1）
    - 序列长度越长，任务越难
    """
    print("=" * 60)
    print("长期依赖任务：RNN 的记忆极限测试")
    print("=" * 60)
    
    def generate_data(n_samples, seq_len):
        """
        生成数据:
        X: 第一个位置是 +1 或 -1，其余为 0
        Y: 等于第一个位置的值
        """
        X = np.zeros((n_samples, seq_len, 1))
        Y = np.zeros((n_samples, 1))
        
        for i in range(n_samples):
            label = np.random.choice([-1, 1])
            X[i, 0, 0] = label  # 只在第一个位置有信号
            Y[i, 0] = label
        
        return torch.FloatTensor(X), torch.FloatTensor(Y)
    
    # 创建 RNN 模型
    class MemoryRNN(nn.Module):
        def __init__(self, hidden_size):
            super().__init__()
            self.rnn = nn.RNN(1, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, 1)
        
        def forward(self, x):
            out, _ = self.rnn(x)
            return torch.tanh(self.fc(out[:, -1, :]))  # 用 tanh 将输出限制在 [-1, 1]
    
    # 测试不同序列长度
    seq_lengths = [5, 10, 20, 50, 100]
    hidden_size = 32
    n_train = 500
    n_test = 100
    n_epochs = 100
    
    results = {}
    
    for seq_len in seq_lengths:
        print(f"\n序列长度 = {seq_len}")
        
        # 生成数据
        X_train, Y_train = generate_data(n_train, seq_len)
        X_test, Y_test = generate_data(n_test, seq_len)
        
        # 创建模型
        torch.manual_seed(42)
        model = MemoryRNN(hidden_size)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = nn.MSELoss()
        
        # 训练
        train_losses = []
        test_accs = []
        
        for epoch in range(n_epochs):
            model.train()
            optimizer.zero_grad()
            
            output = model(X_train)
            loss = criterion(output, Y_train)
            loss.backward()
            
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            train_losses.append(loss.item())
            
            # 测试准确率
            if (epoch + 1) % 10 == 0:
                model.eval()
                with torch.no_grad():
                    pred = model(X_test)
                    # 准确率：预测符号是否正确
                    acc = ((pred > 0) == (Y_test > 0)).float().mean().item()
                    test_accs.append((epoch, acc))
        
        # 最终准确率
        model.eval()
        with torch.no_grad():
            pred = model(X_test)
            final_acc = ((pred > 0) == (Y_test > 0)).float().mean().item()
        
        results[seq_len] = {
            'final_acc': final_acc,
            'train_losses': train_losses,
            'test_accs': test_accs
        }
        
        print(f"  最终准确率: {final_acc*100:.1f}%")
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图：准确率 vs 序列长度
    ax = axes[0]
    accs = [results[s]['final_acc'] * 100 for s in seq_lengths]
    bars = ax.bar(range(len(seq_lengths)), accs, 
                  tick_label=[str(s) for s in seq_lengths],
                  color=['green' if a > 70 else 'orange' if a > 55 else 'red' for a in accs],
                  edgecolor='black')
    ax.axhline(y=50, color='gray', linestyle='--', label='随机猜测 (50%)')
    ax.set_xlabel('序列长度', fontsize=12)
    ax.set_ylabel('准确率 (%)', fontsize=12)
    ax.set_title('RNN 记忆能力 vs 序列长度', fontsize=14)
    ax.set_ylim(0, 105)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # 添加数值标签
    for bar, acc in zip(bars, accs):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                f'{acc:.1f}%', ha='center', fontsize=10)
    
    # 右图：训练曲线
    ax = axes[1]
    for seq_len in seq_lengths:
        ax.plot(results[seq_len]['train_losses'], label=f'seq_len={seq_len}')
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('训练损失', fontsize=12)
    ax.set_title('训练损失曲线', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 结论
    print("\n" + "=" * 60)
    print("结论:")
    print("-" * 60)
    print("• RNN 在短序列上表现良好")
    print("• 序列越长，准确率越接近随机猜测 (50%)")
    print("• 这证明了梯度消失导致 RNN 无法学习长期依赖")
    print("• 解决方案: LSTM 和 GRU（下一章）")
    
    return results


memory_results = long_term_memory_task()

---

## 第四部分：缓解梯度问题的技术

### 4.1 梯度裁剪（Gradient Clipping）

解决梯度爆炸的最直接方法

In [ ]:
def gradient_clipping_demo():
    """
    梯度裁剪演示
    """
    print("=" * 60)
    print("梯度裁剪 (Gradient Clipping)")
    print("=" * 60)
    
    # 模拟一些可能爆炸的梯度
    np.random.seed(42)
    gradients = np.random.randn(100) * np.random.exponential(2, 100)
    
    # 方法 1: 按值裁剪 (Clip by Value)
    max_val = 1.0
    clipped_by_value = np.clip(gradients, -max_val, max_val)
    
    # 方法 2: 按范数裁剪 (Clip by Norm)
    max_norm = 5.0
    grad_norm = np.linalg.norm(gradients)
    if grad_norm > max_norm:
        clipped_by_norm = gradients * (max_norm / grad_norm)
    else:
        clipped_by_norm = gradients.copy()
    
    # 可视化
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 原始梯度
    ax = axes[0, 0]
    ax.bar(range(len(gradients)), gradients, color='blue', alpha=0.7)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.set_title(f'原始梯度 (范数 = {np.linalg.norm(gradients):.2f})', fontsize=12)
    ax.set_xlabel('参数索引')
    ax.set_ylabel('梯度值')
    ax.set_ylim(-10, 10)
    
    # 按值裁剪
    ax = axes[0, 1]
    ax.bar(range(len(clipped_by_value)), clipped_by_value, color='green', alpha=0.7)
    ax.axhline(y=max_val, color='r', linestyle='--', label=f'max = {max_val}')
    ax.axhline(y=-max_val, color='r', linestyle='--')
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.set_title(f'按值裁剪 (max = ±{max_val})', fontsize=12)
    ax.set_xlabel('参数索引')
    ax.set_ylabel('梯度值')
    ax.legend()
    ax.set_ylim(-10, 10)
    
    # 按范数裁剪
    ax = axes[1, 0]
    ax.bar(range(len(clipped_by_norm)), clipped_by_norm, color='orange', alpha=0.7)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.set_title(f'按范数裁剪 (max_norm = {max_norm}, 新范数 = {np.linalg.norm(clipped_by_norm):.2f})', fontsize=12)
    ax.set_xlabel('参数索引')
    ax.set_ylabel('梯度值')
    ax.set_ylim(-10, 10)
    
    # 比较两种方法
    ax = axes[1, 1]
    methods = ['原始', '按值裁剪', '按范数裁剪']
    norms = [np.linalg.norm(gradients), np.linalg.norm(clipped_by_value), np.linalg.norm(clipped_by_norm)]
    bars = ax.bar(methods, norms, color=['blue', 'green', 'orange'], edgecolor='black')
    ax.axhline(y=max_norm, color='r', linestyle='--', label=f'目标范数 = {max_norm}')
    ax.set_title('梯度范数比较', fontsize=12)
    ax.set_ylabel('梯度范数')
    ax.legend()
    
    for bar, norm in zip(bars, norms):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{norm:.2f}', ha='center', fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    # 代码示例
    print("\nPyTorch 梯度裁剪代码:")
    print("-" * 50)
    print("""
# 方法 1: 按范数裁剪（推荐）
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

# 方法 2: 按值裁剪
torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)

# 使用位置: 在 loss.backward() 之后, optimizer.step() 之前
loss.backward()
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # 裁剪
optimizer.step()
""")


gradient_clipping_demo()

### 4.2 其他缓解技术

In [ ]:
def other_techniques():
    """
    其他缓解梯度问题的技术
    """
    print("=" * 60)
    print("缓解梯度问题的其他技术")
    print("=" * 60)
    
    techniques = [
        {
            "name": "1. 权重初始化",
            "description": "使用正交初始化或 Xavier 初始化",
            "code": """
# 正交初始化 (保持梯度稳定)
nn.init.orthogonal_(rnn.weight_hh_l0)

# Xavier 初始化
nn.init.xavier_uniform_(rnn.weight_ih_l0)
""",
            "effect": "防止初始梯度过大或过小"
        },
        {
            "name": "2. 使用 ReLU 而非 tanh",
            "description": "ReLU 的导数是 0 或 1，减少梯度消失",
            "code": """
# 注意: 原版 RNN 用 tanh，但可以用 ReLU 变体
class ReLURNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, 
                         nonlinearity='relu')  # 使用 ReLU
""",
            "effect": "可能导致梯度爆炸，需要配合裁剪使用"
        },
        {
            "name": "3. 批归一化 / 层归一化",
            "description": "归一化隐状态，稳定梯度流动",
            "code": """
class RNNWithLayerNorm(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn_cell = nn.RNNCell(input_size, hidden_size)
        self.layer_norm = nn.LayerNorm(hidden_size)
    
    def forward(self, x, h=None):
        outputs = []
        for t in range(x.size(1)):
            h = self.rnn_cell(x[:, t, :], h)
            h = self.layer_norm(h)  # 层归一化
            outputs.append(h)
        return torch.stack(outputs, dim=1), h
""",
            "effect": "标准化激活值，防止值过大或过小"
        },
        {
            "name": "4. 截断 BPTT",
            "description": "限制反向传播的时间步数",
            "code": """
# 每 20 步截断一次梯度
truncation_len = 20
for i in range(0, seq_len, truncation_len):
    chunk = x[:, i:i+truncation_len, :]
    if h is not None:
        h = h.detach()  # 截断梯度
    output, h = rnn(chunk, h)
""",
            "effect": "限制梯度传播距离，但可能丢失长期信息"
        },
        {
            "name": "5. 残差连接",
            "description": "添加跳跃连接，让梯度直接流过",
            "code": """
class ResidualRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size)
        # 假设 input_size == hidden_size
    
    def forward(self, x, h=None):
        out, h_new = self.rnn(x, h)
        return out + x, h_new  # 残差连接
""",
            "effect": "梯度可以绕过 RNN 直接传回，类似 ResNet"
        }
    ]
    
    for tech in techniques:
        print(f"\n{tech['name']}")
        print("-" * 50)
        print(f"描述: {tech['description']}")
        print(f"效果: {tech['effect']}")
        print(f"代码示例:{tech['code']}")
    
    # 最终解决方案
    print("\n" + "=" * 60)
    print("最终解决方案: LSTM 和 GRU")
    print("=" * 60)
    print("""
上述技术都只是"缓解"梯度问题，真正的"解决"需要改变架构：

LSTM (Long Short-Term Memory):
  - 引入"细胞状态" (Cell State) 作为信息高速公路
  - 使用"门控机制"控制信息流动
  - 梯度可以沿细胞状态直接传递

GRU (Gated Recurrent Unit):
  - LSTM 的简化版本
  - 更少的参数，但效果相近
  
下一章我们将详细讲解 LSTM 的原理和实现！
""")


other_techniques()

---

## 第五部分：总结与展望

### 5.1 本章总结

In [ ]:
def chapter_summary():
    """
    本章知识点总结
    """
    print("=" * 70)
    print("第 4 章总结：梯度消失与梯度爆炸")
    print("=" * 70)
    
    summary = """
    
┌─────────────────────────────────────────────────────────────────────┐
│                         核心问题                                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  BPTT 中的梯度计算：                                                │
│                                                                     │
│    ∂h_t     t                                                       │
│    ──── = ∏ [diag(1-h_i²) · W_hh]                                   │
│    ∂h_k   i=k+1                                                     │
│                                                                     │
│  这是一个指数函数！                                                 │
│                                                                     │
│  • |λ_max(W_hh)| < 1  →  梯度消失  →  无法学习长期依赖              │
│  • |λ_max(W_hh)| > 1  →  梯度爆炸  →  训练不稳定                    │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│                         缓解方法                                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  梯度爆炸（较易解决）:                                              │
│    ✓ 梯度裁剪 (Gradient Clipping)                                   │
│    ✓ 正确的权重初始化                                               │
│                                                                     │
│  梯度消失（较难解决）:                                              │
│    △ 使用 ReLU 激活函数                                             │
│    △ 层归一化 (Layer Normalization)                                 │
│    △ 残差连接                                                       │
│    ★ LSTM / GRU（根本解决方案）                                     │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│                         实验结论                                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. 梯度随时间步指数级衰减/增长                                     │
│  2. RNN 的有效记忆长度约 10-20 步                                   │
│  3. 超过 50 步的长期依赖几乎无法学习                                │
│  4. 这解释了为什么 LSTM 在 1997 年被发明                            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
"""
    
    print(summary)
    
    # 可视化总结
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # 绘制梯度行为示意图
    time_steps = np.arange(0, 100)
    
    # 三种情况
    vanishing = 0.95 ** time_steps
    stable = np.ones_like(time_steps, dtype=float)
    exploding = 1.05 ** time_steps
    
    ax.semilogy(time_steps, vanishing, 'b-', linewidth=2, label='梯度消失 (λ=0.95)')
    ax.semilogy(time_steps, stable, 'g-', linewidth=2, label='理想情况 (λ=1.0)')
    ax.semilogy(time_steps, exploding, 'r-', linewidth=2, label='梯度爆炸 (λ=1.05)')
    
    # 标注区域
    ax.axhspan(1e-10, 1e-5, alpha=0.1, color='blue', label='消失区域')
    ax.axhspan(1e5, 1e10, alpha=0.1, color='red', label='爆炸区域')
    
    ax.set_xlabel('时间步差 (t - k)', fontsize=12)
    ax.set_ylabel('梯度相对大小', fontsize=12)
    ax.set_title('RNN 梯度问题：消失 vs 爆炸', fontsize=14)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.grid(True, alpha=0.3)
    ax.set_ylim(1e-15, 1e15)
    
    plt.tight_layout()
    plt.show()


chapter_summary()

---

## 下一章预告

**第 5 章：LSTM 深度解析**

LSTM 如何解决梯度消失问题？

关键创新：
1. **细胞状态 (Cell State)**: 信息高速公路，梯度可以直接传递
2. **门控机制**: 学习何时记住、何时遗忘、何时输出

下一章我们将：
- 详细理解 LSTM 的每个门的作用
- 用 NumPy 从零实现 LSTM
- 分析 LSTM 如何保持梯度稳定
- 在长期依赖任务上验证 LSTM 的效果

👉 [05_lstm_deep_dive.ipynb](./05_lstm_deep_dive.ipynb)